In [5]:
import numpy as np
import torch
import torch.nn as nn
from sklearn.datasets import load_iris
from sklearn.preprocessing import MinMaxScaler
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer
from qiskit.circuit import ParameterVector

# === 1. เตรียมข้อมูล Iris และ Scaling ===
X, y = load_iris(return_X_y=True)
X = X[y == 0][:50]  # class Setosa
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# === 2. สร้าง VQC ด้วย 8 qubits ===
n_qubits = 8
latent_dim = 8  # Match number of qubits
params = ParameterVector("θ", latent_dim)

def create_vqc(params):
    qc = QuantumCircuit(n_qubits)
    for i in range(n_qubits):
        qc.ry(params[i % latent_dim], i)
    for i in range(n_qubits - 1):
        qc.cx(i, i + 1)
    qc.measure_all()
    return qc

def get_probs(theta, shots=1024):
    qc = create_vqc(params)
    bound = qc.assign_parameters({params[i]: theta[i] for i in range(latent_dim)})
    backend = Aer.get_backend("qasm_simulator")
    tqc = transpile(bound, backend)
    job = backend.run(tqc, shots=shots)
    counts = job.result().get_counts()

    probs = np.zeros(2**n_qubits)
    for bit, count in counts.items():
        idx = int(bit[::-1], 2)  # flip
        probs[idx] = count / shots
    return probs

# === 3. MLP Decoder: 256-dim input → 4 features ===
class Decoder(nn.Module):
    def __init__(self, input_dim=256, output_dim=4):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )
    def forward(self, x):
        return self.net(x)

decoder = Decoder()
optimizer = torch.optim.Adam(decoder.parameters(), lr=0.01)
loss_fn = nn.MSELoss()

# === 4. เทรน VQC + Decoder แบบ Hybrid ===
print("🧠 Training VQC + Classical Decoder...")

for epoch in range(100):
    total_loss = 0.0
    for i in range(len(X_scaled)):
        theta = np.random.uniform(0, 2 * np.pi, latent_dim)
        probs = get_probs(theta)

        x_in = torch.tensor(probs, dtype=torch.float32)
        y_true = torch.tensor(X_scaled[i], dtype=torch.float32)

        y_pred = decoder(x_in)
        loss = loss_fn(y_pred, y_true)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    if epoch % 1 == 0:
        print(f"Epoch {epoch}: Loss = {total_loss:.4f}")

# === 5. Generate new synthetic Iris data ===
print("\n🎉 Generating synthetic data...")
generated_data = []
for _ in range(200):
    theta = np.random.uniform(0, 2 * np.pi, latent_dim)
    probs = get_probs(theta)
    x_in = torch.tensor(probs, dtype=torch.float32)
    with torch.no_grad():
        out = decoder(x_in).numpy()
    generated_data.append(out)

generated_data = scaler.inverse_transform(np.array(generated_data))

# === 6. แสดงผล
import pandas as pd
df = pd.DataFrame(generated_data, columns=["sepal length", "sepal width", "petal length", "petal width"])
df["label"] = "Generated (VQC + MLP)"
print(df.head())

# === Save to CSV
df.to_csv("4_generated_iris.csv", index=False)


🧠 Training VQC + Classical Decoder...
Epoch 0: Loss = 3.6093
Epoch 10: Loss = 2.4506
Epoch 20: Loss = 2.2735
Epoch 30: Loss = 2.1855
Epoch 40: Loss = 2.1576
Epoch 50: Loss = 2.1408
Epoch 60: Loss = 2.1698
Epoch 70: Loss = 2.2230
Epoch 80: Loss = 2.1678
Epoch 90: Loss = 2.1684

🎉 Generating synthetic data...
   sepal length  sepal width  petal length  petal width                  label
0      5.014321     3.425645      1.465074     0.250401  Generated (VQC + MLP)
1      5.014321     3.425645      1.465074     0.250401  Generated (VQC + MLP)
2      5.014321     3.425645      1.465074     0.250401  Generated (VQC + MLP)
3      5.014321     3.425645      1.465074     0.250401  Generated (VQC + MLP)
4      5.014321     3.425645      1.465074     0.250401  Generated (VQC + MLP)
